In [2]:
import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import utils as np_utils
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils

In [3]:
import tensorflow as tf

In [4]:
def extract_n_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [4]:
n_instances = 500
n_teachers = 10

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )


169017344/169001437 [==============================] - 2s 0us/step


In [5]:
teacher_data_x.shape, teacher_data_y.shape

((10, 500, 32, 32, 3), (10, 500, 100))

In [6]:
student_data_x.shape,student_data_y.shape

((500, 32, 32, 3), (500, 100))

In [7]:
## train the teacher models
# train the teacher models according to cifar100 dataset in tensorflow
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ],run_eagerly=True)

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    #model.save( '/content/drive/MyDrive/Pate_one_class/10_teac/teacher_model_{}.h5'.format( i ) )


teacher 0
test accuracy: 0.0706000030040741
teacher 1
test accuracy: 0.056699998676776886
teacher 2
test accuracy: 0.06469999998807907
teacher 3
test accuracy: 0.05220000073313713
teacher 4
test accuracy: 0.06080000102519989
teacher 5
test accuracy: 0.053599998354911804
teacher 6
test accuracy: 0.05530000105500221
teacher 7
test accuracy: 0.06340000033378601
teacher 8
test accuracy: 0.05380000174045563
teacher 9
test accuracy: 0.07129999995231628


training the student model

In [8]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [9]:
# label the data
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [10]:
labels.shape

(10, 500, 100)

In [11]:
student_data_x.shape

(500, 32, 32, 3)

In [12]:
len(student_data_x)

500

In [13]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [14]:
student_data_x.shape,student_data_y.shape,votes.shape

((500, 32, 32, 3), (500, 100), (500, 100))

In [15]:
ll = np.argmax( votes, axis=1 )

In [16]:
ll.shape

(500,)

In [17]:
student_data_y.shape,keras.utils.np_utils.to_categorical( ll,num_classes=100 ).shape

((500, 100), (500, 100))

In [18]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy for the student model for considering 10 teaches:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy for the student model for considering 10 teaches: 0.006000000052154064


20 teachers

In [19]:
n_instances = 500 
n_teachers = 20

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )


In [20]:
# Training 
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    #model.save( '/content/drive/MyDrive/Pate_one_class/20_teac/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.059700001031160355
teacher 1
test accuracy: 0.05590000003576279
teacher 2
test accuracy: 0.052400000393390656
teacher 3
test accuracy: 0.060499999672174454
teacher 4
test accuracy: 0.06849999725818634
teacher 5
test accuracy: 0.0632999986410141
teacher 6
test accuracy: 0.06129999831318855
teacher 7
test accuracy: 0.0649000033736229
teacher 8
test accuracy: 0.0771000012755394
teacher 9
test accuracy: 0.05939999967813492
teacher 10
test accuracy: 0.054499998688697815
teacher 11
test accuracy: 0.0778999999165535
teacher 12
test accuracy: 0.05849999934434891
teacher 13
test accuracy: 0.05400000140070915
teacher 14
test accuracy: 0.06989999860525131
teacher 15
test accuracy: 0.054099999368190765
teacher 16
test accuracy: 0.06509999930858612
teacher 17
test accuracy: 0.061900001019239426
teacher 18
test accuracy: 0.0625
teacher 19
test accuracy: 0.05570000037550926


In [21]:
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [22]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [23]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy for the student model for considering 20 teaches:', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy for the student model for considering 20 teaches: 0.017999999225139618


50 teachers

In [5]:
n_instances = 500 
n_teachers = 50

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( -1, 32, 32, 3)
x_test = x_test.reshape( -1, 32, 32, 3)

x_train, y_train = extract_n_classes( x_train, y_train ,n=100,cat_y=False)

y_train = keras.utils.np_utils.to_categorical( y_train )
y_test = keras.utils.np_utils.to_categorical( y_test )

# shuffle data
idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the teacher data
teacher_data_x = np.array([ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])
teacher_data_y = np.array([ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_teachers ) ])

# gather the student data
student_data_x = np.array(x_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ] )
student_data_y = np.array(y_train[ n_teachers * n_instances : ( n_teachers + 1 ) * n_instances ]  )


In [6]:
# Training 
def get_model():
  # Create the model
  model = Sequential()
  model.add(Conv2D(16, kernel_size=1, activation='relu'))
  model.add(Conv2D(16, kernel_size=3, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='softmax'))


  model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )

  return model
    

# list of teacher models
teacher_models = [ get_model() for i in range( n_teachers ) ]

for i, (model, x, y) in enumerate( zip( teacher_models, teacher_data_x, teacher_data_y ) ):
    print( 'teacher', i )
    model.fit( x, y, batch_size = 32, epochs = 10, verbose = 0 )
    print( 'test accuracy:', model.evaluate( x_test, y_test, verbose = 0 )[ 1 ] )
    #model.save( '/content/drive/MyDrive/Pate_one_class/50_teac/teacher_model_{}.h5'.format( i ) )

teacher 0
test accuracy: 0.06480000168085098
teacher 1
test accuracy: 0.05130000039935112
teacher 2
test accuracy: 0.06530000269412994
teacher 3
test accuracy: 0.06159999966621399
teacher 4
test accuracy: 0.06620000302791595
teacher 5
test accuracy: 0.05389999970793724
teacher 6
test accuracy: 0.06639999896287918
teacher 7
test accuracy: 0.05169999971985817
teacher 8
test accuracy: 0.06679999828338623
teacher 9
test accuracy: 0.05249999836087227
teacher 10
test accuracy: 0.05169999971985817
teacher 11
test accuracy: 0.05249999836087227
teacher 12
test accuracy: 0.05920000001788139
teacher 13
test accuracy: 0.0714000016450882
teacher 14
test accuracy: 0.05990000069141388
teacher 15
test accuracy: 0.05660000070929527
teacher 16
test accuracy: 0.07360000163316727
teacher 17
test accuracy: 0.060100000351667404
teacher 18
test accuracy: 0.05510000139474869
teacher 19
test accuracy: 0.061400000005960464
teacher 20
test accuracy: 0.059700001031160355
teacher 21
test accuracy: 0.05770000070333

In [7]:
labels = np.array([ teacher.predict( student_data_x ) for teacher in teacher_models ])

In [8]:
# preform the voting
votes = np.zeros( ( student_data_x.shape[ 0 ], 100 ), dtype=np.float )
for i in range( len( student_data_x ) ):
  for j in range( n_teachers ):
    label = np.argmax( labels[ j ][ i ] )
    votes[ i, label ] += 1
  # add the noise per class
  for j in range( 10 ):
    votes[ i, j ] += np.random.laplace(loc=0.0, scale=5 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [9]:
student_data_y = keras.utils.np_utils.to_categorical( np.argmax( votes, axis=1 ),num_classes=100 )

# train model
student_model = get_model()
print( 'training student model' )
student_model.fit( x, y, epochs=16, verbose=0 )
print( 'test accuracy for the student model for considering 50 teaches :', student_model.evaluate( student_data_x, student_data_y, verbose=0 )[ 1 ] )

training student model
test accuracy for the student model for considering 50 teaches : 0.0560000017285347
